In [1]:
import numpy as np
from scipy.special import ndtr
from scipy.stats import norm, ncx2
from scipy.optimize import minimize
from numpy.polynomial.hermite import hermfit, hermval, hermder
import copy

#### Spot rates to ZCB


In [14]:
def spot_rates_to_zcb(T,spot_rate):
    M = len(T)
    p = np.zeros([M])
    for i in range(0,M):
        if T[i] < 1e-8:
            p[i] = 1
        else:
            p[i] = np.exp(-spot_rate[i]*T[i])
    return p

#Example:
T = [1/4, 2/4, 3/4, 4/4]
spot_rate = [ 1.42669978,  0.4462871 ,  0.14048069, -0.        ]

print(spot_rates_to_zcb(T,spot_rate))

[0.7 0.8 0.9 1. ]


Input:
\begin{equation}
\begin{aligned}
T &= \text{time to maturity} \\
\text{spot\_rate}&=\text{Spot rate}
\end{aligned}
\end{equation}

Output:
\begin{equation}
\begin{aligned}
p &= \text{ZCB price} 
\end{aligned}
\end{equation}

Formula:
\begin{equation}
\begin{aligned}
p(t,T)=\exp(-R(t,T)\cdot(T-t))
\end{aligned}
\end{equation}


#### ZCB to spot rates

In [13]:
def zcb_to_spot_rates(T,p):
    M = len(T)
    R = np.zeros([M])
    for i in range(0,M):
        if T[i] < 1e-8:
            R[i] = 0
        else:
            R[i] = -np.log(p[i])/T[i]
    return R

#Example:
T = [1/4, 2/4, 3/4, 4/4]
p = [0.7, 0.8, 0.9, 1]

print(zcb_to_spot_rates(T,p))

[ 1.42669978  0.4462871   0.14048069 -0.        ]


Input:
\begin{equation}
\begin{aligned}
T &= \text{time to maturity} \\
p &= \text{ZCB price} 
\end{aligned}
\end{equation}

Output:
\begin{equation}
\begin{aligned}
R &= \text{spot rate} 
\end{aligned}
\end{equation}

Formula:
\begin{equation}
\begin{aligned}
R(t,T) &= -\frac{\log p(t,T)}{(T-t)}
\end{aligned}
\end{equation}


#### ZCB to forward rates

In [22]:
def zcb_to_forward_rates(T,p,horizon = 1):
    # horizon = 0 corresponds to instantaneous forward rates
    M = len(T)
    f = np.zeros([M])
    if horizon == 0:
        f[0] = (np.log(p[0])-np.log(p[1]))/(T[1]-T[0])
        f[-1] = (np.log(p[-2])-np.log(p[-1]))/(T[-1]-T[-2])
        m = 1
        while m < M - 1.5: #1.5 is abritrary and might as well be 1.0001, as far as I understand...
            f[m] = (np.log(p[m-1])-np.log(p[m+1]))/(T[m+1]-T[m-1])
            m += 1
    elif 0 < horizon:
        m = horizon
        while m < M - 0.5:
            f[m] = (np.log(p[m-horizon])-np.log(p[m]))/(T[m]-T[m-horizon])
            m += 1
    return f

#Example:
T = [1/4, 2/4, 3/4, 4/4]
p = [0.7, 0.8, 0.9, 1]

print(zcb_to_forward_rates(T,p, horizon=1))
print(zcb_to_forward_rates(T,p, horizon=0))

[ 0.         -0.53412557 -0.47113214 -0.42144206]
[-0.53412557 -0.50262886 -0.4462871  -0.42144206]


Input:
\begin{equation}
\begin{aligned}
T &= \text{time to maturity} \\
p &= \text{ZCB price} \\
horizon&=\text{start date of loan}
\end{aligned}
\end{equation}

Output:
\begin{equation}
\begin{aligned}
f=\text{forward rates of return}
\end{aligned}
\end{equation}

Formula:
\begin{equation}
\begin{aligned}
R(t;S,T)=-\frac{\log p(t,T)-\log p(t,S)}{(T-S)}
\end{aligned}
\end{equation}


#### ZCB to forward LIBOR rates

In [24]:
def zcb_to_forward_LIBOR_rates(T,p,horizon = 1):
    M = len(T)
    f = np.zeros([M])
    i = horizon
    #ERROR: Need to include what happens when horizon = 0
    while i < M - 0.5:
        f[i] = (p[i-horizon]-p[i])/(p[i]*(T[i]-T[i-horizon]))
        i += 1
    return f

#Example:
T = [1/4, 2/4, 3/4, 4/4]
p = [0.7, 0.8, 0.9, 1]

print(zcb_to_forward_LIBOR_rates(T,p, horizon=1))


[ 0.         -0.5        -0.44444444 -0.4       ]


Input:
\begin{equation}
\begin{aligned}
T &= \text{time to maturity} \\
p &= \text{ZCB price} \\
horizon&=\text{start date of loan}
\end{aligned}
\end{equation}

Output:
\begin{equation}
\begin{aligned}
f=\text{forward rates of return}
\end{aligned}
\end{equation}

Formula (unsure):
\begin{equation}
\begin{aligned}
\text{Forward LIBOR Rate} = \frac{p(t, S) - p(t, T)}{p(t, T) \cdot (T - S)}
\end{aligned}
\end{equation}


#### Utility function used in many of the following functions:

In [85]:
def value_in_list_returns_I_idx(value,list):
    output = False, None
    for i, item in enumerate(list):
        if abs(value-item) < 1e-6:
            output = True, i
            break
    return output

#Example:
example_list = [1/8, 2/8, 3/8, 4/8, 5/8, 6/8, 7/8, 8/8]
example_value = [2/8, 4/8, 6/8, 8/8]

for i in range(0,len(example_value)):
    I_fix, idx_fix = value_in_list_returns_I_idx(example_value[i], example_list)
    if I_fix is True:
        print(example_list[idx_fix])

0.25
0.5
0.75
1.0


In essence, the function only returns values that are in both lists.

#### ZCB to accrual factor (also known as present value of a basis point)

In [127]:
def zcb_to_accrual_factor(T_n,T_N,fixed_freq,T,p):
    if fixed_freq == "quarterly":
        p_fix = np.zeros([int(T_N-T_n)*4])
        T_fix = np.array([T_n + i*0.25 for i in range(1,int(T_N-T_n)*4 + 1)])
    elif fixed_freq == "semiannual":
        p_fix = np.zeros([int(T_N-T_n)*2])
        T_fix = np.array([T_n + i*0.5 for i in range(1,int(T_N-T_n)*2 + 1)])
    elif fixed_freq == "annual":
        p_fix = np.zeros([int(T_N-T_n)])
        T_fix = np.array([T_n + i*0.25 for i in range(1,int(T_N-T_n) + 1)])
    S = 0
    for i in range(0,len(T_fix)):
        I_fix, idx_fix = value_in_list_returns_I_idx(T_fix[i],T)
        if I_fix is True:
            T_fix[i] = T[idx_fix]
            p_fix[i] = p[idx_fix]
            if i == 0:
                S += T_fix[i]*p_fix[i]
            else:
                S += (T_fix[i]-T_fix[i-1])*p_fix[i]


    return S

#Example:
T = [1/4, 2/4, 3/4, 4/4]
p = [0.7, 0.8, 0.9, 1]

S = zcb_to_accrual_factor(0,1,"quarterly",T,p)

print(S)


0.85


Input:
\begin{equation}
\begin{aligned}
T_n &= \text{Start time of the accrual period} \\
T_N &= \text{End time of the accrual period} \\
\text{fixed\_freq} &= \text{Frequency of accrual} \\
T &= \text{time to maturity} \\
p &= \text{ZCB price}
\end{aligned}
\end{equation}

Output:
\begin{equation}
\begin{aligned}
S=\text{Accrual factor}
\end{aligned}
\end{equation}

Formula:
\begin{equation}
\begin{aligned}
S_{n}^{k}(t) &=\sum_{i=n+1}^{k}\alpha_{i}p(t,T_{i}) \\
\alpha_{i} &=T_{i}-T_{i-1}
\end{aligned}
\end{equation}

where $n$ is the start period and $k$ is the end period.


#### ZCB to par swap rate

In [129]:
def zcb_to_par_swap_rate(T_n,T_N,fixed_freq,T,p):
    I_n, idx_n = value_in_list_returns_I_idx(T_n,T)
    I_N, idx_N = value_in_list_returns_I_idx(T_N,T)
    D = p[idx_n]-p[idx_N]
    S = zcb_to_accrual_factor(T_n,T_N,fixed_freq,T,p)
    R = D/S
    return R

#Example:
T = [0, 1/4, 2/4, 3/4, 4/4]
p = [0.5, 0.7, 0.8, 0.9, 1]

zcb_to_par_swap_rate(0,1,"quarterly",T,p)

-0.5882352941176471

Input:
\begin{equation}
\begin{aligned}
T_n &= \text{Start time of the accrual period} \\
T_N &= \text{End time of the accrual period} \\
\text{fixed\_freq} &= \text{Frequency of accrual} \\
T &= \text{time to maturity} \\
p &= \text{ZCB price}
\end{aligned}
\end{equation}

Output:
\begin{equation}
\begin{aligned}
R=\text{Par swap rate}
\end{aligned}
\end{equation}

Formula:
\begin{equation}
\begin{aligned}
R_{n}^{N}(t)=\frac{p_{n}(t)-p_{N}(t)}{\sum_{i=n+1}^{N}\alpha_{i}p_{i}(t)}
\end{aligned}
\end{equation}

where $n$ is the start period and $N$ is the end period. Note that the denominator is the accrual rate.


#### ZCB price for vasicek

In [144]:
def zcb_price_vasicek(r0,a,b,sigma,tau):
    if type(tau) == int or type(tau) == float:
        B = (1/a)*(1-np.exp(-a*tau))
        A = (B-tau)*(a*b-0.5*sigma**2)/(a**2)-(sigma**2*B)/(4*a)
        p = np.exp(A-r0*B)
    elif type(tau) == tuple or type(tau) == list or type(tau) == np.ndarray:
        M = len(tau)
        p = np.zeros([M])
        for i in range(0,M):
            B = (1/a)*(1-np.exp(-a*tau[i]))
            A = (B-tau[i])*(a*b-0.5*sigma**2)/(a**2)-(sigma**2*B)/(4*a)
            #ERROR: Why is it not B**2????
            p[i] = np.exp(A-r0*B)
    else:
        print(f"tau not a recognized type")
        p = False
    return p

#Example:
r0, a, b, sigma_vasicek = 0.025, 0.5, 0.025, 0.02
alpha = 0.25
T_max = 10

M = int(round(T_max/alpha))

T = np.array([i*alpha for i in range(0,M+1)])

print(zcb_price_vasicek(r0, a, b, sigma_vasicek, T))

[1.         0.99336228 0.98611486 0.97834796 0.97014137 0.96156554
 0.9526825  0.94354674 0.93420609 0.9247024  0.91507227 0.90534762
 0.89555625 0.88572234 0.87586689 0.86600808 0.85616164 0.84634112
 0.83655822 0.82682295 0.81714389 0.80752836 0.79798256 0.78851175
 0.77912033 0.76981198 0.76058975 0.75145611 0.74241308 0.73346224
 0.72460482 0.71584171 0.70717356 0.69860077 0.69012352 0.68174184
 0.67345559 0.6652645  0.6571682  0.64916622 0.64125799]


Input:
\begin{equation}
\begin{aligned}
r_0 &= \text{Initial short rate} \\
a &= \text{Speed of reversion} \\
b &= \text{Long-term mean level} \\
\sigma &= \text{Volatility} \\
\tau &= \text{Time to maturity}
\end{aligned}
\end{equation}

Output:
\begin{equation}
p = \text{Price of the ZCB}
\end{equation}

Formula:
\begin{equation}
p(t, T) = e^{A(t,T)-B(t,T)r(t)}
\end{equation}
where
\begin{equation}
\begin{aligned}
A(t,T)&=\frac{\left[B(t,T)-(T-t)\right]\left(ab-\frac{1}{2}\sigma^{2}\right)}{a^{2}}-\frac{\sigma^{2}B^{2}(t,T)}{4a}\\
B(t, T) &= \frac{1}{a}\left[1 - e^{-a(T-t)}\right]
\end{aligned}
\end{equation}

#### Spot rate for vasicek

In [142]:
def spot_rate_vasicek(r0,a,b,sigma,tau):
    if type(tau) == int or type(tau) == float:
        B = (1/a)*(1-np.exp(-a*tau))
        A = (B-tau)*(a*b-0.5*sigma**2)/(a**2)-(sigma**2*B)/(4*a)
        if tau < 1e-6:
            r = 0
        elif tau >= 1e-6:
            r = (-A+r0*B)/tau
    elif type(tau) == tuple or type(tau) == list or type(tau) == np.ndarray:
        M = len(tau)
        r = np.zeros([M])
        for i in range(0,M):
            B = (1/a)*(1-np.exp(-a*tau[i]))
            A = (B-tau[i])*(a*b-0.5*sigma**2)/(a**2)-(sigma**2*B)/(4*a)
            if tau[i] < 1e-6:
                r[i] = 0
            else:
                r[i] = (-A+r0*B)/tau[i]
    else:
        print(f"tau not a recognized type")
        r = False
    return r

#Example:
r0, a, b, sigma_vasicek = 0.025, 0.5, 0.025, 0.02
alpha = 0.25
T_max = 10

M = int(round(T_max/alpha))

T = np.array([i*alpha for i in range(0,M+1)])

print(spot_rate_vasicek(r0, a, b, sigma_vasicek, T))

[0.         0.02663941 0.02796488 0.02918651 0.03031347 0.03135404
 0.03231573 0.03320536 0.03402911 0.03479259 0.03550089 0.03615865
 0.03677008 0.037339   0.0378689  0.03836294 0.03882402 0.03925477
 0.03965759 0.04003467 0.04038802 0.04071945 0.04103064 0.04132313
 0.0415983  0.04185744 0.04210172 0.04233222 0.04254992 0.04275575
 0.04295051 0.04313499 0.04330989 0.04347586 0.04363349 0.04378334
 0.04392591 0.04406168 0.04419108 0.04431451 0.04443234]


Input: Same as above

Output: Spot rate

Formula:

\begin{equation}
R(t,T)=\frac{-A(t,T)+rB(t,T)}{T-t}
\end{equation}

where $A$ and $B$ are the same as above.

#### Forward rate vasicek

In [145]:
def forward_rate_vasicek(r0,a,b,sigma,tau):
    if type(tau) == int or type(tau) == float:
        B = (1/a)*(1-np.exp(-a*tau))
        B_T = np.exp(-a*tau)
        if tau < 1e-6:
            f = 0
        elif tau >= 1e-6:
            f = (1-B_T)*(a*b-0.5*sigma**2)/(a**2) + (sigma**2*B*B_T)/(2*a) + r0*B_T
    elif type(tau) == tuple or type(tau) == list or type(tau) == np.ndarray:
        M = len(tau)
        f = np.zeros([M])
        for i in range(0,M):
            B = (1/a)*(1-np.exp(-a*tau[i]))
            B_T = np.exp(-a*tau[i])
            if tau[i] < 1e-6:
                f[i] = 0
            else:
                f[i] = (1-B_T)*(a*b-0.5*sigma**2)/(a**2) + (sigma**2*B*B_T)/(2*a) + r0*B_T
    else:
        print(f"tau not a recognized type")
        f = False
    return f

#Example:
r0, a, b, sigma_vasicek = 0.025, 0.5, 0.025, 0.02
alpha = 0.25
T_max = 10

M = int(round(T_max/alpha))

T = np.array([i*alpha for i in range(0,M+1)])

print(forward_rate_vasicek(r0, a, b, sigma_vasicek, T))

[0.         0.02792653 0.03049084 0.03273954 0.03471288 0.03644568
 0.03796812 0.03930641 0.04048335 0.04151881 0.04243012 0.04323241
 0.04393892 0.04456125 0.04510953 0.04559268 0.0460185  0.04639386
 0.04672477 0.04701654 0.04727382 0.04750071 0.04770082 0.04787732
 0.048033   0.04817033 0.04829148 0.04839836 0.04849265 0.04857584
 0.04864924 0.048714   0.04877115 0.04882157 0.04886605 0.04890531
 0.04893995 0.04897052 0.04899749 0.04902129 0.0490423 ]


Input: Same as above

Output: Forward rate

Formula:

\begin{equation}
f(t,T)=\frac{(1-B_{T})(ab-\frac{1}{2}\sigma^{2})}{a^{2}}+\frac{\sigma^{2}BB_{T}}{2a}+rB_{t}
\end{equation}

where $A$ and $B$ are the same as above and $B_{t}=\exp(-a\cdot(T-t))$



#### Spot rate bump

In [ ]:
def spot_rate_bump(T_bump,size_bump,T,R_input,p_input):
    R, p = R_input.copy(), p_input.copy()
    if type(T_bump) == int or type(T_bump) == float or type(T_bump) == np.float64:
        I_bump, idx_bump = value_in_list_returns_I_idx(T_bump,T)
        R[idx_bump] = R[idx_bump] + size_bump
        p[idx_bump] = np.exp(-R[idx_bump]*T_bump)
    elif type(T_bump) == tuple or type(T_bump) == list or type(T_bump) == np.ndarray:
        if type(size_bump) == int or type(size_bump) == float or type(size_bump) == np.float64:
            for i in range(0,len(T_bump)):
                I_bump, idx_bump = value_in_list_returns_I_idx(T_bump[i],T)
                R[idx_bump] = R[idx_bump] + size_bump
                p[idx_bump] = np.exp(-R[idx_bump]*T_bump[i])
        elif type(size_bump) == tuple or type(size_bump) == list or type(size_bump) == np.ndarray:
            for i in range(0,len(T_bump)):
                I_bump, idx_bump = value_in_list_returns_I_idx(T_bump[i],T)
                R[idx_bump] = R[idx_bump] + size_bump[i]
                p[idx_bump] = np.exp(-R[idx_bump]*T_bump[i])
    return R, p


